## Fine-tuning a model on a text classification task
This notebook will show to fine-tune one of the 🤗 Transformers model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).
<br><br>
The **GLUE Benchmark** is a group of nine classification tasks on sentences or pairs of sentences which are:

- **CoLA** (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not. It is a dataset containing sentences labeled grammatically correct or not.
- **MNLI** (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- **MRPC** (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- **QNLI** (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- **QQP** (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- **RTE** (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- **SST-2** (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- **STS-B** (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- **WNLI** (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)
<br><br>

Among them, this notebook will use The **CoLA** Dataset.

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 8.6MB/s 
     |████████████████████████████████| 901kB 30.9MB/s 
     |████████████████████████████████| 3.3MB 38.9MB/s 
     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 112kB 11.9MB/s 
     |████████████████████████████████| 245kB 11.7MB/s 


In [2]:
import datasets

import random
import pandas as pd
import numpy as np

from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# datasets : 1.5.0  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1
print(f'datasets : {datasets.__version__}  |  pd : {pd.__version__}  |  np : {np.__version__}  |  transformers : {transformers.__version__}')

datasets : 1.6.0  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1


We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## 1. Loading the dataset & metric
We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

In [4]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = datasets.load_dataset("glue", actual_task)
metric = datasets.load_metric('glue', actual_task)

print('\n>>> actual_task :', actual_task)
print('\n>>> type of metric :', type(metric))
print('\n>>> dataset object :', dataset)
print('\n>>> sample data :', dataset['train'][0])

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.




>>> actual_task : cola

>>> type of metric : <class 'datasets_modules.metrics.glue.e4606ab9804a36bcd5a9cebb2cb65bb14b6ac78ee9e6d5981fa679a495dd55de.glue.Glue'>

>>> dataset object : DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

>>> sample data : {'idx': 0, 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}


In [5]:
# show random sample of a dataset
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), k=num_examples)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,idx,label,sentence
0,7680,acceptable,John believes Bill to have cooked the rice.
1,5692,acceptable,The student loved his phonology readings.
2,5995,acceptable,Calvin has not eaten his dinner.
3,3268,acceptable,Bill is drying a lot of clothes on the line.
4,3406,acceptable,Maggie hurried through the museum.


In [6]:
# compute score with metric
fake_preds = np.random.randint(0, 2, size=(16,))
fake_labels = np.random.randint(0, 2, size=(16,))

fake_preds, fake_labels, metric.compute(predictions=fake_preds, references=fake_labels)

(array([1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]),
 {'matthews_correlation': -0.06666666666666667})

The metric object only computes the proper metrics associated to the task, which are:

- for **CoLA** : Matthews Correlation Coefficient
- for **MNLI**(matched or mismatched) : Accuracy
- for **MRPC** : Accuracy and F1 score
- for **QNLI** : Accuracy
- for **QQP** : Accuracy and F1 score
- for **RTE** : Accuracy
- for **SST-2** : Accuracy
- for **STS-B** : Pearson Correlation Coefficient and Spearman's_Rank_Correlation_Coefficient
- for **WNLI** : Accuracy

## 2. Preprocessing the data


In [7]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


CPU times: user 345 ms, sys: 42.4 ms, total: 388 ms
Wall time: 2.92 s


In [8]:
# tokenize sample sentences
batch_encoded = tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

print(type(batch_encoded))
for k, v in batch_encoded.items():
  print(k, '-', v)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
input_ids - [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102]
attention_mask - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
# generate dict for saving names of columns containing the sentences
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

# check if the dict works correctly
sent1_key, sent2_key = task_to_keys[task]
if sent2_key is None:
  print(f'Sentence :', dataset['train'][0][sent1_key])
else:
  print(f'Sentence 1 :', dataset['train'][0][sent1_key])
  print(f'Sentence 2 :', dataset['train'][0][sent2_key])

Sentence : Our friends won't buy this analysis, let alone the next one we propose.


In [10]:
# function to preprocess text - tokenize & truncate
def preprocess(examples):
  if sent2_key is None:
    return tokenizer(examples[sent1_key], truncation=True)
  else:
    return tokenizer(examples[sent1_key], examples[sent2_key], truncation=True)


# check if the preprocessor works properly
for k, v in preprocess(dataset['train'][:5]).items():
  print(f"'{k}' :")
  for lst in v:
    print('\t', lst)

'input_ids' :
	 [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]
	 [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102]
	 [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102]
	 [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102]
	 [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]
'attention_mask' :
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
%%time
# apply preprocess function to all sentences in dataset
# set batched as True to encode the texts by batches together (about 5x faster)
dataset_encoded = dataset.map(preprocess, batched=True)

for k, v in dataset_encoded['train'][0].items():
  print(f'>>> {k} - {v}')


>>> attention_mask - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
>>> idx - 0
>>> input_ids - [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]
>>> label - 1
>>> sentence - Our friends won't buy this analysis, let alone the next one we propose.
CPU times: user 869 ms, sys: 14.9 ms, total: 884 ms
Wall time: 720 ms
